<a href="https://colab.research.google.com/github/Akanksha-cell-max/Advanced-Artificial-Intelligence/blob/main/Block_worldProblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches


# Define predicate classes
class ON:
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __repr__(self):
        return f"ON({self.X}, {self.Y})"


class ONTABLE:
    def __init__(self, X):
        self.X = X

    def __repr__(self):
        return f"ONTABLE({self.X})"


class CLEAR:
    def __init__(self, X):
        self.X = X

    def __repr__(self):
        return f"CLEAR({self.X})"


class ARMEMPTY:
    def __repr__(self):
        return "ARMEMPTY()"


class HOLDING:
    def __init__(self, X):
        self.X = X

    def __repr__(self):
        return f"HOLDING({self.X})"


class BlocksWorldVisualizer:
    def __init__(self, world_state, goal_state=None):
        self.world_state = world_state
        self.goal_state = goal_state
        self.fig, self.ax = plt.subplots()

    def draw(self):
        self.ax.clear()
        self.ax.set_xlim(-2, 10)
        self.ax.set_ylim(-1, 4)
        self.ax.set_aspect('equal', 'box')

        self.draw_state(self.world_state, offset=0, label="Initial State")
        if self.goal_state:
            self.draw_state(self.goal_state, offset=6, label="Goal State")

        self.ax.axis('off')
        plt.show()

    def draw_state(self, world_state, offset, label):
        self.ax.text(offset + 1, 3, label, ha='center', va='center', fontsize=14, fontweight='bold')

        on_table = [pred.X for pred in world_state if isinstance(pred, ONTABLE)]
        for i, block in enumerate(on_table):
            self.ax.add_patch(patches.Rectangle((i * 2 + offset, 0), 1, 1, edgecolor="black", facecolor="skyblue"))
            self.ax.text(i * 2 + offset + 0.5, 0.5, block, color="black", ha="center", va="center", fontsize=12)

        on = [pred for pred in world_state if isinstance(pred, ON)]
        for pred in on:
            self.ax.add_patch(patches.Rectangle((on_table.index(pred.Y) * 2 + offset, 1), 1, 1, edgecolor="black", facecolor="lightgreen"))
            self.ax.text(on_table.index(pred.Y) * 2 + offset + 0.5, 1.5, pred.X, color="black", ha="center", va="center", fontsize=12)

        holding = [pred.X for pred in world_state if isinstance(pred, HOLDING)]
        for block in holding:
            self.ax.add_patch(patches.Circle((offset + 3, 2), 0.5, edgecolor="black", facecolor="orange"))
            self.ax.text(offset + 3, 2, block, color="black", ha="center", va="center", fontsize=12)


class GoalStackPlannerWithVisualization:
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.visualizer = BlocksWorldVisualizer(initial_state, goal_state)

    def get_steps(self):
        steps = self.plan()
        print("Initial state and Goal state:")
        self.visualizer.draw()

        for step in steps:
            print(f"Executing: {step['action']} {step['block']}" +
                  (f" from {step['from']}" if 'from' in step else "") +
                  (f" to {step['to']}" if 'to' in step else ""))

            self.visualizer.world_state = self.apply_step(step)
            self.visualizer.draw()

        return steps

    def apply_step(self, step):
        world_state = self.visualizer.world_state.copy()

        for delete_pred in step["delete"]:
            if delete_pred in world_state:
                world_state.remove(delete_pred)

        for add_pred in step["add"]:
            if add_pred not in world_state:
                world_state.append(add_pred)

        return world_state

    def plan(self):
        """ Demonstrates Pickup, Putdown, Stack, and Unstack operations """
        return [
            {"action": "Unstack", "block": "B", "from": "A",
             "delete": [ON("B", "A"), CLEAR("B")],
             "add": [HOLDING("B"), CLEAR("A")]},

            {"action": "Putdown", "block": "B",
             "delete": [HOLDING("B")],
             "add": [ONTABLE("B"), CLEAR("B")]},

            {"action": "Pickup", "block": "C",
             "delete": [ONTABLE("C"), CLEAR("C")],
             "add": [HOLDING("C")]},

            {"action": "Stack", "block": "C", "to": "A",
             "delete": [HOLDING("C"), CLEAR("A")],
             "add": [ON("C", "A"), CLEAR("C")]}
        ]


# Define initial and goal states
initial_state = [
    ON("B", "A"),
    ONTABLE("A"), ONTABLE("C"), ONTABLE("D"),
    CLEAR("B"), CLEAR("C"), CLEAR("D"),
    ARMEMPTY()
]

goal_state = [
    ON("B", "D"), ON("C", "A"),
    ONTABLE("D"), ONTABLE("A"),
    CLEAR("B"), CLEAR("C"),
    ARMEMPTY()
]

goal_stack = GoalStackPlannerWithVisualization(initial_state=initial_state, goal_state=goal_state)
steps = goal_stack.get_steps()
